# Einleitung

In diesem Notebook vergleichen wir verschiedene Ansätze, um sicherzustellen, dass unser Bot die am besten passenden Antworten generiert.

Die 5 Fragen, die verglichen sollen folgende Eigenschaften besitzen:
- Die Fragen sollen Fakultät übergreifend sein
- Die Fragen sollen TH-spezifisch sein
- Die Fragen sollen TH-Intranet-spezifisch sein
- Die Fragen sollen Realitätsnah sein

Die 5 Fragen, die verglichen werden sind:

1. Wie ist die Email Adresse von Professor Gallwitz?
2. Was soll ich beachten, wenn ich eine Prüfung anmelden will?
3. Welche voraussetzungen, muss ich für den Mater Studiengang erfüllen?
4. Welche Professoren gibt es an der Fakultät Soziale Arbeit?
5. Wann und was muss ich im IT-Projekt machen?

## LLM Model: Vicuna groß

Nun werden fogende Parameter verglichen.

| LLM Modell    | Kontext Variation | Word Embedding Model | Frage 1 | Antwort 1 | Frage 2 | Antwort 2 | Frage 3 | Antwort 3 | Frage 4 | Antwort 4 | Frage 5 | Antwort 5 | Durchschnitt |
|---------------|-------------------|----------------------|---------|-----------|---------|-----------|---------|-----------|---------|-----------|---------|-----------|--------------|
| Vicuna groß   | 1 Dokument        | a(MiniLM)            | 2       |           | 4       |           | 5       |           | 8       |           | 7       |           | 5.2          |
| Vicuna groß   | 1 Dokument        | b(xy)                 | 10      |           | 10      |           | 10      |           | 10      |           | 10      |           | 10           |
| Vicuna groß   | 1 Dokument        | c(TF-IDF)             |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 1 Dokument        | d(MiniLM*TF-IDF)      |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 1 Dokument        | e(xy*TF-IDF)          |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 5 Dokumente       | a                     |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 5 Dokumente       | b                     |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 5 Dokumente       | c                     |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 5 Dokumente       | d                     |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 5 Dokumente       | e                     |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 10 Dokumente      | a                     |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 10 Dokumente      | b                     |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 10 Dokumente      | c                     |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 10 Dokumente      | d                     |         |           |         |           |         |           |         |           |         |           | 0            |
| Vicuna groß   | 10 Dokumente      | e                     |         |           |         |           |         |           |         |           |         |           | 0            |


Nun soll ein script ersteltt werden, der die Antwort Spalten generiert.
Nacher werden die Antworten manuell bewertet von allen Projekt Teilnehmer.

In [ ]:
import sys
sys.path.append('..')
sys.path.append('../..')
from scrap.query_crafter import construct_prompt
from scrap.embedding_algorithms.tdIdfDistance import get_most_similar_articles_tf_idf
from scrap.embedding_algorithms.question_embedding_MiniLM import get_most_similar_articles_MiniLM
from LLM.ServerCommunicator import server_get_answer

In [ ]:
def get_most_similar_articles_for_specified_embediing_model(embedding_model,question,document_amount):
    
    embeddings_model_dict = {
        'MiniLM': get_most_similar_articles_MiniLM,
        'TF-IDF': get_most_similar_articles_tf_idf
    }
     
    get_most_similar_articles = embeddings_model_dict.get(embedding_model)
    result = get_most_similar_articles(question,document_amount)
    
    return result

In [ ]:
async def get_anwser_from_model(model_id,prompt):

    model_dict = {
        'vicuna_70b': server_get_answer,
        'vicuna_13b': get_most_similar_articles_tf_idf,
        'llama_13b': get_most_similar_articles_tf_idf

    }
    get_anwser = model_dict.get(model_id)
    anwser = await get_anwser(prompt)

    return anwser

Make sure to forward the port for the vicuna_70b model.
To do so, copy the template into the terminal:  ssh -N -L localhost:8080:localhost:8087 <musterm123>@141.75.89.6 and enter your password afterwards.


In [ ]:
async def generate_response(model_id,document_amount,embedding_model,question):
    anwser=""
    documents = get_most_similar_articles_for_specified_embediing_model(embedding_model,question,document_amount)
    prompt = construct_prompt(documents, question) 
    anwser = await get_anwser_from_model(model_id,prompt)
    
    return anwser

In [ ]:
models= ["vicuna_70b","vicuna_13b","llama_13b"]
document_amounts=[1,5,10]
embeddings_models=["MiniLM","TF-IDF"]
questions=["Wie ist die Email Adresse von Professor Gallwitz?",
           "Was soll ich beachten, wenn ich eine Prüfung anmelden will?",
           "Welche voraussetzungen, muss ich für den Mater Studiengang erfüllen?",
           "Welche Professoren gibt es an der Fakultät Soziale Arbeit?",
           "Wann und was muss ich im IT-Projekt machen?"]

for model_id in models:
    for document_amount in document_amounts:
        for embeddings_model in embeddings_models:
            for question in questions:
                response = generate_response(model_id, document_amount, embeddings_model, question)
                print(f"Model: {model_id}, Docs: {document_amount}, Embeddings: {embeddings_model}, Question: {question}")
                print(f"Response: {response}")
                print("-" * 30)

In [ ]:
await generate_response("vicuna_70b",1,"MiniLM","Wie ist die Email Adresse von Professor Gallwitz?")

In [ ]:
import pandas as pd

# Geben Sie den Dateipfad zu Ihrer Excel-Datei an
excel_file_path = 'C:\\Users\\lizab\\Desktop\\TH\\Semester6\\ItProjekt\\Evaluation.xlsx'

# Laden Sie die Excel-Datei in einen Pandas DataFrame
df = pd.read_excel(excel_file_path, sheet_name='Sheet1')

# Modifizieren Sie die Zellen nach Bedarf
# Beispiel: Setzen Sie den Wert 42 in die Zelle in der ersten Zeile und ersten Spalte
df.at[0, 'Spaltenname'] = "halloichbins"

# Speichern Sie die Änderungen zurück in die Excel-Datei
df.to_excel(excel_file_path, sheet_name='Sheet1', index=False)


In [ ]:
import pandas as pd

# Erstellen Sie eine leere DataFrame
columns = ["Model", "Document Amount", "Embeddings Model", "Question", "Response"]
df = pd.DataFrame(columns=columns)

# Ihre Daten
models = ["vicuna_70b"]
document_amounts = [1, 5, 10]
embeddings_models=["MiniLM","TF-IDF"]
questions=["Wie ist die Email Adresse von Professor Gallwitz?",
           "Was soll ich beachten, wenn ich eine Prüfung anmelden will?",
           "Welche voraussetzungen, muss ich für den Mater Studiengang erfüllen?",
           "Welche Professoren gibt es an der Fakultät Soziale Arbeit?",
           "Wann und was muss ich im IT-Projekt machen?"]

data = []
for model in models:
    for document_amount in document_amounts:
        for embeddings_model in embeddings_models:
            for question in questions:
                response = await generate_response(model, document_amount, embeddings_model, question)
                data.append([model, document_amount, embeddings_model, question, response])

df = pd.concat([df, pd.DataFrame(data, columns=columns)], ignore_index=True)

# Speichern Sie die DataFrame in eine Excel-Datei
df.to_excel('output.xlsx', index=False)
